# Basic Example

Example usage of the time series aggregation module (tsam).

This notebook demonstrates:
1. Basic k-means aggregation
2. Hierarchical aggregation with extreme periods
3. Advanced aggregation with segmentation

Import pandas and the relevant time series aggregation class

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import pandas as pd

# Configure Plotly for sphinx/nbsphinx output
import plotly.io as pio

import tsam
from tsam import ClusterConfig, ExtremeConfig, SegmentConfig

pio.renderers.default = "notebook"

# Ensure results directory exists
RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(exist_ok=True)

### Input data 

Read in time series from testdata.csv with pandas

In [ ]:
raw = pd.read_csv("testdata.csv", index_col=0)

Show a slice of the dataset

In [ ]:
raw.head()

Show the shape of the raw input data: 4 types of timeseries (GHI, Temperature, Wind and Load) for every hour in a year

In [ ]:
raw.shape

Plot the original temperature data as a heatmap

In [ ]:
# Use tsam's built-in heatmap plotting
tsam.plot.heatmap(raw, column="T", period_hours=24, title="Original Temperature")

Use the `aggregate()` function with k-means clustering for eight typical days.

In [ ]:
result_kmeans = tsam.aggregate(
    raw,
    n_clusters=8,
    period_hours=24,
    cluster=ClusterConfig(method="kmeans"),
)

Access the typical periods from the result object

In [ ]:
cluster_representatives = result_kmeans.cluster_representatives
cluster_representatives.head()

Show shape of typical periods: 4 types of timeseries for 8*24 hours

In [ ]:
print(f"Shape: {cluster_representatives.shape}")
print(
    f"Periods: {result_kmeans.n_clusters}, Timesteps per period: {result_kmeans.n_timesteps_per_period}"
)

Save typical periods to .csv file

In [ ]:
cluster_representatives.to_csv(RESULTS_DIR / "testperiods_kmeans.csv")

Reconstruct the original time series based on the typical periods

In [ ]:
reconstructed = result_kmeans.reconstruct()

Plot the repredicted data

In [ ]:
# K-means reconstructed temperature heatmap
tsam.plot.heatmap(
    reconstructed,
    column="T",
    period_hours=24,
    title="K-means Reconstructed Temperature",
)

As seen, they days with the minimal temperature are excluded. In case that they are required they can be added to the aggregation as follow.

### Hierarchical aggregation including extreme periods

Use hierarchical clustering with extreme period preservation. This ensures the day with the minimum temperature and maximum load are included.

In [ ]:
result_hier = tsam.aggregate(
    raw,
    n_clusters=8,
    period_hours=24,
    cluster=ClusterConfig(method="hierarchical"),
    extremes=ExtremeConfig(
        method="new_cluster",
        min_value=["T"],  # Preserve day with minimum temperature
        max_value=["Load"],  # Preserve day with maximum load
    ),
)

Create the typical periods

In [ ]:
cluster_representatives = result_hier.cluster_representatives
cluster_representatives.head()

The aggregation can also be evaluated by indicators

In [ ]:
# View accuracy metrics
print(result_hier.accuracy)
print("\nRMSE per column:")
print(result_hier.accuracy.rmse)

Save typical periods to .csv file

In [ ]:
cluster_representatives.to_csv(RESULTS_DIR / "testperiods_hierarchical.csv")

Repredict the original time series based on the typical periods

In [ ]:
reconstructed_extremes = result_hier.reconstruct()

Plot repredicted data

In [ ]:
# Hierarchical with extremes reconstructed temperature heatmap
tsam.plot.heatmap(
    reconstructed_extremes,
    column="T",
    period_hours=24,
    title="Hierarchical + Extremes Reconstructed Temperature",
)

Now also the days with the minimal temperature are integrated into the typical periods.

### Advanced aggregation method

Combining hierarchical clustering with segmentation (reduced temporal resolution) and distribution-preserving representation.

In [ ]:
result_advanced = tsam.aggregate(
    raw,
    n_clusters=24,
    period_hours=24,
    cluster=ClusterConfig(
        method="hierarchical",
        representation="distribution_minmax",
    ),
    segments=SegmentConfig(n_segments=8),
)

In [ ]:
reconstructed_advanced = result_advanced.reconstruct()

In [ ]:
# Advanced method reconstructed temperature heatmap
tsam.plot.heatmap(
    reconstructed_advanced,
    column="T",
    period_hours=24,
    title="Advanced Method Reconstructed Temperature",
)

### Comparison of the aggregations 
It was shown for the temperature, but both times all four time series have been aggregated. Therefore, we compare here also the duration curves  of the electrical load for the original time series, the aggregation with k-mean, and the hierarchical aggregation including peak periods.

In [ ]:
# Duration curve comparison using built-in function
tsam.plot.compare(
    {
        "Original": raw,
        "8 typ days": reconstructed,
        "8 typ days + peak": reconstructed_extremes,
        "24 typ days + 8 seg": reconstructed_advanced,
    },
    column="Load",
    plot_type="duration_curve",
)

Or as unsorted time series for an example week

In [ ]:
# Time slice comparison - Load
tsam.plot.compare(
    {
        "Original": raw,
        "8 typ days": reconstructed,
        "8 typ days + peak": reconstructed_extremes,
        "24 typ days + 8 seg": reconstructed_advanced,
    },
    column="Load",
    plot_type="time_slice",
    start="20100210",
    end="20100218",
)

In [ ]:
# Time slice comparison - GHI
tsam.plot.compare(
    {
        "Original": raw,
        "8 typ days": reconstructed,
        "8 typ days + peak": reconstructed_extremes,
        "24 typ days + 8 seg": reconstructed_advanced,
    },
    column="GHI",
    plot_type="time_slice",
    start="20100210",
    end="20100218",
    title="Time Slice Comparison - Solar Irradiance (Feb 10-18)",
)